In [1]:
import ee
ee.Authenticate()
ee.Initialize()

In [2]:
import pandas as pd
import numpy as np
import datetime as dt
import os

import matplotlib.pyplot as plt

In [37]:
def delta_factor(var, dailyMC, dailyMT, datesC, datesT):

    deltas = np.zeros((1,12))

    for im in range(1,13):
            
        monAct = im

        indxsC  = np.where([(dt.month==monAct)for dt in datesC])
        indxsT  = np.where([(dt.month==monAct)for dt in datesT])

        # Obs
        tempDailyC  = dailyMC[indxsC].mean()
        tempDailyMT = dailyMT[indxsT].mean()
        print(tempDailyC, tempDailyMT)
        
        
        if var == 'pr': 
            if tempDailyC != 0:
                deltas[0][im-1] = (tempDailyMT) / tempDailyC #(tempDailyMT - tempDailyC) / tempDailyC#(tempDailyMT) / tempDailyC
            else:
                deltas[0][im-1] = 1.0
        else:
            deltas[0][im-1] = tempDailyMT - tempDailyC
    
    #print(deltas[0])
    return deltas[0]

In [27]:
def dates_dt(date_years):
    start_date = dt.date(date_years[0],    1, 1)
    end_date   = dt.date(date_years[1]+1, 1, 1)
    dates      = [start_date + dt.timedelta(n) for n in range(int ((end_date - start_date).days))]     
    return dates

In [28]:
def ee_array_to_data(arr, list_of_bands):
    """Transforms client-side ee.Image.getRegion array to pandas.DataFrame."""
    data = pd.DataFrame(arr)

    # Rearrange the header.
    headers = data.iloc[0]
    data    = pd.DataFrame(data.values[1:], columns=headers)

    # Remove rows without data inside.
    data = data[['id','longitude', 'latitude', 'time', *list_of_bands]]

    # Convert the data to numeric values.
    for band in list_of_bands:
        data[band] = pd.to_numeric(data[band], errors='coerce')

    # Convert the time field into a datetime.
    data['datetime'] = pd.to_datetime(data['time'], unit='ms')

    # Keep the columns of interest.
    data = data[['time','datetime',  *list_of_bands]]
    
    return data

In [29]:
def extract_GEE_NEX_GDDP(var, date, GCM_EE_model_names, scenario, estaciones_lista, name_stations):

    # Access GEE data:"NASA/GDDP-CMIP6" and  extract series (nearest grid)
    GCM_EE     = ee.ImageCollection("NASA/GDDP-CMIP6")    
    estaciones = ee.FeatureCollection(estaciones_lista)
    
    # Set date target
    start_date = f'{date[0]}-01-01'
    end_date = f'{date[1]+1}-01-01'
    
    GEE_EE_data = []
    
    # Filter and storage data in dataframe
    for i in range(len(GCM_EE_model_names)):    
        GCM_EE_filter = GCM_EE.select(var).filter(
                        ee.Filter.And(ee.Filter.eq('scenario', scenario),
                        ee.Filter.eq('model', GCM_EE_model_names[i]),
                        ee.Filter.date(start_date, end_date)))
        
        extraction = GCM_EE_filter.getRegion(estaciones, 100).getInfo() 
        arr_data = ee_array_to_data(extraction,[var])
        if var == 'pr':
            data_arr = pd.DataFrame(np.reshape(arr_data[var].values,(len(estaciones_lista),-1))).transpose()*86400
        data_arr.columns=name_stations
        data_arr['date'] = arr_data.datetime.unique()
        data_arr = data_arr.set_index('date')
        GEE_EE_data.append(data_arr)
    
    return GEE_EE_data

In [30]:
def output_folder_directory(var, outputh_folder_path):
    # Create output folte if it does not exist
    output_path = os.path.join(outputh_folder_path, var)
    isExist = os.path.exists(output_path)

    if isExist == False:
        os.makedirs(output_path) 
    
    os.chdir(output_path)
    return output_path

In [31]:
def delta_NEXGDDP(path_coords, path_data, var, step_time, ref_date, calibr_date, target_date, scenario, GCM_EE_model_names, output_path):
    print('Processing...')
    # Read data
    estaciones_data = pd.read_csv(path_coords, encoding='latin8')
    observed_data   = pd.read_csv(path_data, encoding='latin8')    

    # Coordinates and name stations
    estaciones_lista=[]
    for row in range(len(estaciones_data.index)):
        points=ee.Feature(ee.Geometry.Point(estaciones_data.longitud[row],
                                            estaciones_data.latitude[row]),
                         {'name':(estaciones_data.station_name[row])})
        estaciones_lista.append(points)

    name_stations = estaciones_data.station_name.to_list()        
 
    # Set dates   
    datesO = dates_dt(ref_date)
    datesC = dates_dt(calibr_date)
    datesT = dates_dt(target_date)
    
    # Historical
    scenario_hist = 'historical'
    GEE_EE_hist_data = extract_GEE_NEX_GDDP(var, calibr_date, GCM_EE_model_names, scenario_hist, estaciones_lista, name_stations)
    
    # Future
    scenario_target = scenario
    GEE_EE_target_data = extract_GEE_NEX_GDDP(var, target_date, GCM_EE_model_names, scenario, estaciones_lista, name_stations)
        
    output_folder = output_folder_directory(var, output_path)
    os.chdir(output_folder)
        
    future_date_serie = pd.date_range(f'{ref_date[0]}-01-01', f'{ref_date[1]}-12-31', freq=step_time)
    delta_series_sd   = pd.DataFrame({'date': pd.to_datetime(future_date_serie)})
    delta_series_sd = delta_series_sd.set_index('date') 
    
    delta_factors = pd.DataFrame({'Month': np.arange(1,13,1)})
    
    for i in range(len(GCM_EE_model_names)):
        
        historical_data = GEE_EE_hist_data[i]
        future_data     = GEE_EE_target_data[i]

        for station in name_stations:
            
            dailyO  = np.array(observed_data[f'{station}'])
            
            dailyProjected = np.zeros_like(dailyO)
            
            dailyMC = np.array(historical_data[f'{station}'])
            dailyMT = np.array(future_data[f'{station}'])

            deltas = delta_factor(var, dailyMC, dailyMT, datesC, datesT)
            for im in range(1,13):
                index_month = delta_series_sd.index.month == im
                if var == 'pr':
                    dailyProjected[index_month] = dailyO[index_month]*deltas[im-1]
                else:
                    dailyProjected[index_month] = dailyO[index_month]+deltas[im-1]    
            
            delta_series_sd[f'{station}'] = dailyProjected  
            delta_factors[f'{station}'] = deltas 
            os.chdir(output_folder)
        
        delta_factors.round(3).to_csv(f'{var}_{GCM_EE_model_names[i]}_{scenario}_{target_date[0]}_{target_date[1]}_delta.csv'
                                ,encoding='latin8'
                                , index=False)    
        delta_series_sd.round(3).to_csv(f'dc_{var}_{GCM_EE_model_names[i]}_{scenario}_{target_date[0]}_{target_date[1]}.csv'
                                ,encoding='latin8'
                                )      
        
        
        print('Done : ', f'{var}_{GCM_EE_model_names[i]}_{scenario}_{target_date[0]}_{target_date[1]}')

Name of the CMIP6 model.  
It is one of: 'ACCESS-CM2', 'ACCESS-ESM1-5', 'BCC-CSM2-MR', 'CESM2', 'CESM2-WACCM', 'CMCC-CM2-SR5', 'CMCC-ESM2', 'CNRM-CM6-1', 'CNRM-ESM2-1', 'CanESM5', 'EC-Earth3', 'EC-Earth3-Veg-LR', 'FGOALS-g3', 'GFDL-CM4', 'GFDL-ESM4', 'GISS-E2-1-G', 'HadGEM3-GC31-LL', 'HadGEM3-GC31-MM', 'IITM-ESM', 'INM-CM4-8', 'INM-CM5-0', 'IPSL-CM6A-LR', 'KACE-1-0-G', 'KIOST-ESM', 'MIROC-ES2L', 'MIROC6', 'MPI-ESM1-2-HR', 'MPI-ESM1-2-LR', 'MRI-ESM2-0', 'NESM3', 'NorESM2-LM', 'NorESM2-MM', 'TaiESM1', 'UKESM1-0-LL'.

In [36]:
var = 'pr'
path_coords = r"C:\Users\Carolina\CC_EMT\estaciones_rocha.csv"
path_data   = r"C:\Users\Carolina\CC_EMT\pcp_rocha.csv"
step_time   = 'd'
ref_date    = [1970,2021]
calibr_date = [1950,2014]
target_date = [2036,2065]
scenario    = 'ssp585'
GCM_EE_model_names = ['MIROC6']
output_path = r"D:\00_OUTPUT_CC_EMT"

delta_NEXGDDP(path_coords, path_data, var, step_time, ref_date, calibr_date, target_date, scenario, GCM_EE_model_names, output_path)

Processing...
4.58116482941268 4.128268552182689
4.332261396042244 4.845703539959619
3.0986791395594864 3.805749822730884
0.8283992684374368 1.0248538697932545
0.28068259487424335 0.3262845511865831
0.2992986171701025 0.41615453550210896
0.3232174147038766 0.5363061282653482
0.48174111530694114 0.3649852526925843
0.8131795192946246 0.8856613799097772
1.2177490074548916 1.2647994479462823
1.9028593469804775 1.9549286771986945
3.4492826568070867 3.7450152822069622
[0.90113949 1.11851597 1.22818454 1.23714966 1.16246806 1.39043254
 1.65927362 0.75763775 1.0891339  1.03863722 1.02736373 1.08573743]
Done :  pr_MIROC6_ssp585_2036_2065


In [35]:
var = 'pr'
path_coords = r"C:\Users\Carolina\CC_EMT\estaciones_ichurata.csv"
path_data   = r"C:\Users\Carolina\CC_EMT\pcp_ichurata.csv"
step_time   = 'd'
ref_date    = [1976,2022]
calibr_date = [1950,2014]
target_date = [2036,2065]
scenario    = 'ssp585'
GCM_EE_model_names = ['GFDL-ESM4']
output_path = r"D:\00_OUTPUT_CC_EMT"

delta_NEXGDDP(path_coords, path_data, var, step_time, ref_date, calibr_date, target_date, scenario, GCM_EE_model_names, output_path)

Processing...
2.8926661623602112 2.808608491873348
2.8265303275809535 2.584892792876464
1.7828320546975303 1.6573419358234875
0.5361146000851966 0.658577333406356
0.006481828513958494 0.007236931522774901
0.0 0.0
0.0 0.0
0.0 0.0
0.32984712353265 0.26809079802478664
0.6911940740770371 0.8080014443153378
1.3923156228963938 1.2608659414809154
2.344299378898854 2.14467153641518
[0.97094111 0.9145109  0.92961192 1.22842641 1.11649537 1.
 1.         1.         0.81277288 1.16899359 0.90558916 0.91484541]
Done :  pr_GFDL-ESM4_ssp585_2036_2065
